In [2]:
# Cell 1: Imports & config
import os
import io
import warnings
from datetime import datetime

import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

OUT_DIR = "output"
os.makedirs(OUT_DIR, exist_ok=True)

# Optional: silence some FutureWarnings while debugging
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# Cell 2: Helper - download stock data (robust to multiindex columns)
def download_stock(ticker, period="max", interval="1d", auto_adjust=False):
    """
    Returns DataFrame with Date column and flattened columns (Open, High, Low, Close, Adj Close, Volume).
    Also saves CSV to output/<TICKER>_stock.csv
    """
    # ensure consistent options
    stock = yf.download(ticker, period=period, interval=interval, auto_adjust=auto_adjust, progress=False)
    if stock.empty:
        raise RuntimeError(f"No data returned from yfinance for {ticker}.")
    stock = stock.reset_index()

    # If columns are a MultiIndex (sometimes returned), flatten them
    if isinstance(stock.columns, pd.MultiIndex):
        stock.columns = ["_".join([str(x) for x in col]).strip("_") for col in stock.columns.values]

    # Normalize column names: common names are Date, Open, High, Low, Close, Adj Close, Volume
    # Some flattening may produce 'Close_TSLA' etc — try to map
    cols = {c: c for c in stock.columns}
    # lower-case mapping helpers
    for c in stock.columns:
        lc = c.lower()
        if lc.endswith("close") and "close" not in cols:
            cols[c] = "Close"
        if lc.endswith("adj close") or "adjclose" in lc.replace(" ", ""):
            cols[c] = "Adj Close"
        if lc.endswith("open"):
            cols[c] = "Open"
        if lc.endswith("high"):
            cols[c] = "High"
        if lc.endswith("low"):
            cols[c] = "Low"
        if lc.endswith("volume"):
            cols[c] = "Volume"
        if lc == "date":
            cols[c] = "Date"

    stock = stock.rename(columns=cols)
    # ensure Date column exists
    if "Date" not in stock.columns:
        # attempt to find a datetime-like column
        for c in stock.columns:
            if pd.api.types.is_datetime64_any_dtype(stock[c]):
                stock = stock.rename(columns={c: "Date"})
                break

    stock.to_csv(os.path.join(OUT_DIR, f"{ticker}_stock.csv"), index=False)
    return stock

In [4]:
# Cell 3: Robust Macrotrends revenue scraper
def scrape_revenue_from_macrotrends(ticker_slug, company_name):
    """
    Scrape quarterly revenue table from Macrotrends for given slug/company.
    Returns DataFrame with columns ['Date','Revenue'] where Revenue is numeric (USD).
    Saves CSV to output/<company_name>_revenue.csv
    """
    url = f"https://www.macrotrends.net/stocks/charts/{ticker_slug}/{company_name}/revenue"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    r = requests.get(url, headers=headers, timeout=20)
    if r.status_code != 200:
        raise RuntimeError(f"Request to Macrotrends failed: status {r.status_code}")

    text = r.text

    # First attempt: parse all HTML tables using pandas.read_html (fast, often works)
    try:
        tables = pd.read_html(io.StringIO(text))
    except Exception as e:
        tables = []

    revenue_df = None
    # Search tables for columns that contain both 'Date' and 'Revenue'
    for df in tables:
        cols = [str(c).lower() for c in df.columns.astype(str)]
        if any("date" in c for c in cols) and any("revenue" in c for c in cols):
            revenue_df = df.copy()
            break

    # Fallback 1: sometimes Macrotrends uses different column labels like 'Quarter' or 'Historical Quarterly Revenue'
    if revenue_df is None:
        for df in tables:
            cols = [str(c).lower() for c in df.columns.astype(str)]
            if any("quarter" in c for c in cols) and any("revenue" in c for c in cols):
                revenue_df = df.copy()
                break

    # Fallback 2: try searching the page HTML for the table fragment containing 'Historical Quarterly Revenue'
    if revenue_df is None:
        soup = BeautifulSoup(text, "html.parser")
        # find any table that contains the word 'Revenue' in its text
        html_tables = soup.find_all("table")
        for t in html_tables:
            txt = t.get_text(separator=" ").lower()
            if "revenue" in txt and "date" in txt:
                try:
                    candidate = pd.read_html(io.StringIO(str(t)))[0]
                    cols = [str(c).lower() for c in candidate.columns.astype(str)]
                    if any("date" in c for c in cols) and any("revenue" in c for c in cols):
                        revenue_df = candidate.copy()
                        break
                except Exception:
                    continue

    if revenue_df is None:
        # final fallback: try to find numbers via regex (less structured) — helpful for debugging/inspection
        raise ValueError(
            "Could not find a revenue table on Macrotrends. "
            "Possible reasons: page structure changed, or content loaded by JavaScript. "
            "Open the URL in a browser and inspect the table. URL: " + url
        )

    # Normalize revenue_df: pick the date-like and revenue-like columns
    revenue_df.columns = [str(c).strip() for c in revenue_df.columns]
    date_col = None
    rev_col = None
    for c in revenue_df.columns:
        lc = c.lower()
        if "date" in lc or "quarter" in lc or "period" in lc:
            date_col = c
        if "revenue" in lc:
            rev_col = c
    if date_col is None or rev_col is None:
        # try best-effort
        date_col = revenue_df.columns[0]
        rev_col = revenue_df.columns[1] if len(revenue_df.columns) > 1 else revenue_df.columns[-1]

    revenue_df = revenue_df[[date_col, rev_col]].copy()
    revenue_df.columns = ["Date", "Revenue"]

    # Clean Revenue: remove dollar signs, commas, parens, and text like 'ttm' etc.
    revenue_df["Revenue"] = (
        revenue_df["Revenue"]
        .astype(str)
        .str.replace(r"[\$,()]", "", regex=True)
        .str.replace(r"[A-Za-z]", "", regex=True)
        .str.strip()
    )
    revenue_df = revenue_df[revenue_df["Revenue"] != ""]
    revenue_df["Revenue"] = pd.to_numeric(revenue_df["Revenue"], errors="coerce")

    # Parse Date: try pd.to_datetime, else parse quarter strings like 'Q1 2020'
    def parse_quarter_or_date(s):
        s = str(s).strip()
        # Try direct parse
        try:
            dt = pd.to_datetime(s, errors="coerce")
            if not pd.isna(dt):
                return dt
        except Exception:
            pass
        # Quarter style "Q1 2020" or "2020 Q1"
        import re
        m = re.search(r"q([1-4])\s*[, -/]?\s*(\d{4})", s, flags=re.I)
        if m:
            q = int(m.group(1))
            y = int(m.group(2))
            month = {1: 3, 2: 6, 3: 9, 4: 12}[q]
            # use last day of quarter approximate: pick month end day 1 (ok for sorting/plot)
            return pd.Timestamp(year=y, month=month, day=1)
        # try extracting year-month or year
        m2 = re.search(r"(\d{4})[-/](\d{1,2})", s)
        if m2:
            y = int(m2.group(1)); mo = int(m2.group(2))
            return pd.Timestamp(year=y, month=mo, day=1)
        m3 = re.search(r"(\d{4})", s)
        if m3:
            return pd.Timestamp(year=int(m3.group(1)), month=1, day=1)
        return pd.NaT

    revenue_df["Date"] = revenue_df["Date"].apply(parse_quarter_or_date)
    revenue_df = revenue_df.dropna(subset=["Revenue"])
    revenue_df = revenue_df.sort_values("Date").reset_index(drop=True)
    revenue_df.to_csv(os.path.join(OUT_DIR, f"{company_name}_revenue.csv"), index=False)
    return revenue_df

In [5]:
# Cell 4: download Tesla & show head
print("Downloading Tesla (TSLA) stock data...")
tesla_data = download_stock("TSLA", period="max", interval="1d", auto_adjust=False)
print("\nTesla head():")
display(tesla_data.head())


Tesla head():


,Date,Adj Close,Close_TSLA,High_TSLA,Low_TSLA,Open_TSLA,Volume_TSLA
0,2010-06-29,1.592667,1.592667,1.666667,1.169333,1.266667,281494500
1,2010-06-30,1.588667,1.588667,2.028000,1.553333,1.719333,257806500
2,2010-07-01,1.464000,1.464000,1.728000,1.351333,1.666667,123282000
3,2010-07-02,1.280000,1.280000,1.540000,1.247333,1.533333,77097000
4,2010-07-06,1.074000,1.074000,1.333333,1.055333,1.333333,103003500


In [6]:
# Cell 5: scrape Tesla revenue and show tail
print("Scraping Tesla revenue from Macrotrends...")
try:
    tesla_revenue = scrape_revenue_from_macrotrends("tesla", "tesla")
    print("\nTesla revenue tail():")
    display(tesla_revenue.tail())
except Exception as e:
    print("Error scraping Tesla revenue:", e)
    # If this fails, you can show the page (open URL in browser) and inspect the table structure.

Scraping Tesla revenue from Macrotrends...

Tesla revenue tail():


,Date,Revenue


In [7]:
# Cell 6: download GME & show head
print("Downloading GameStop (GME) stock data...")
gme_data = download_stock("GME", period="max", interval="1d", auto_adjust=False)
print("\nGME head():")
display(gme_data.head())


GME head():


,Date,Adj Close,Close_GME,High_GME,Low_GME,Open_GME,Volume_GME
0,2002-02-13,1.691667,2.51250,2.51500,2.38125,2.40625,76216000
1,2002-02-14,1.683250,2.50000,2.54875,2.48125,2.54375,11021600
2,2002-02-15,1.674834,2.48750,2.50625,2.46250,2.50000,8389600
3,2002-02-19,1.607504,2.38750,2.47500,2.34375,2.47500,7410400
4,2002-02-20,1.662210,2.46875,2.46875,2.38125,2.40000,6892800


In [8]:
# Cell 7: scrape GME revenue and show tail
print("Scraping GameStop revenue from Macrotrends...")
try:
    gme_revenue = scrape_revenue_from_macrotrends("gamestop", "gamestop")
    print("\nGME revenue tail():")
    display(gme_revenue.tail())
except Exception as e:
    print("Error scraping GameStop revenue:", e)

Scraping GameStop revenue from Macrotrends...

GME revenue tail():


,Date,Revenue


In [9]:
# Cell 8: plotting helper (same as before, but with a small safety for columns)
def make_graph(stock_data, revenue_data, title):
    stock_df = stock_data.copy()
    rev_df = revenue_data.copy()

    # prefer Close, fallback to Adj Close, else choose numeric column
    if "Close" in stock_df.columns:
        price_col = "Close"
    elif "Adj Close" in stock_df.columns:
        price_col = "Adj Close"
    else:
        # pick rightmost numeric column besides Date
        numcols = [c for c in stock_df.columns if c != "Date" and pd.api.types.is_numeric_dtype(stock_df[c])]
        price_col = numcols[-1] if numcols else stock_df.columns[-1]

    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.08,
        subplot_titles=(f"{title} — Stock Price", f"{title} — Quarterly Revenue")
    )

    fig.add_trace(go.Scatter(x=stock_df["Date"], y=stock_df[price_col], name="Close Price"), row=1, col=1)
    fig.add_trace(go.Bar(x=rev_df["Date"], y=rev_df["Revenue"], name="Revenue"), row=2, col=1)

    fig.update_yaxes(title_text="USD", row=1, col=1)
    fig.update_yaxes(title_text="USD", row=2, col=1)
    fig.update_layout(height=700, title_text=title, showlegend=False)

    fname = os.path.join(OUT_DIR, f"{title.replace(' ', '_')}_dashboard.html")
    fig.write_html(fname)
    print("Saved dashboard to:", fname)
    return fig

In [10]:
# Cell 9: build dashboards (if data exists)
if "tesla_data" in globals() and "tesla_revenue" in globals():
    print("Creating Tesla dashboard...")
    fig_tesla = make_graph(tesla_data, tesla_revenue, "Tesla (TSLA) Stock vs Revenue")
else:
    print("Tesla data/revenue not available; skipping Tesla dashboard.")

if "gme_data" in globals() and "gme_revenue" in globals():
    print("Creating GameStop dashboard...")
    fig_gme = make_graph(gme_data, gme_revenue, "GameStop (GME) Stock vs Revenue")
else:
    print("GME data/revenue not available; skipping GME dashboard.")

Creating Tesla dashboard...
Saved dashboard to: output\Tesla_(TSLA)_Stock_vs_Revenue_dashboard.html
Creating GameStop dashboard...
Saved dashboard to: output\GameStop_(GME)_Stock_vs_Revenue_dashboard.html


In [11]:
# Cell 10: Reminder / submission text file
instructions = """Submission instructions (what to screenshot):
1) Show the code cell that downloads Tesla stock and the printed head() output.
2) Show the web-scraping code cell for Tesla revenue and the printed tail() output.
3) Show the code cell that downloads GME stock and the printed head() output.
4) The web-scraping code cell for GME revenue and the printed tail() output.
5) Open the saved HTML dashboards in output/ and screenshot each dashboard window.
6) Upload your notebook to GitHub or Watson Studio and paste the link in the assignment portal.
"""
with open(os.path.join(OUT_DIR, "README_submission_instructions.txt"), "w") as f:
    f.write(instructions)
print("Wrote README and ready. Output files are in the 'output/' folder.")

Wrote README and ready. Output files are in the 'output/' folder.
